In [1]:
import time
import pickle
import pandas as pd
import autosklearn.regression
from sklearn.metrics import mean_squared_error
pd.pandas.set_option('display.max_columns',None)
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("Number of latex.csv")
df = df[['Humidity', 'Wind', 'Tempature', 'Total(latex)(dry Milk)']]
df = df.dropna()

with open('primary_df.pickle', 'wb') as file:
    pickle.dump(df, file, protocol=pickle.HIGHEST_PROTOCOL)


y = df['Total(latex)(dry Milk)']

# Select the features (all columns except the target variable)
X = df.drop('Total(latex)(dry Milk)', axis=1)
 

In [3]:
# Split the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
print(X_train.shape)
print(y_train.shape)

(87, 3)
(87,)


In [5]:
start_time = time.time()

automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=900,
    per_run_time_limit=600,
    tmp_folder="/tmp/autosklearn_regression",
    n_jobs = 5,
    memory_limit = 2500
)
automl.fit(X_train, y_train, dataset_name="latex_percision")

print("--- %s seconds ---" % (time.time() - start_time))

[WARNING] [2023-09-11 18:55:47,468:Client-AutoML(1):latex_percision] Capping the per_run_time_limit to 449.0 to have time for a least 2 models in each process.
--- 896.2879598140717 seconds ---


In [6]:
print(automl.leaderboard())

          rank  ensemble_weight         type      cost  duration
model_id                                                        
226          1             0.20  extra_trees  0.485663  0.962245
265          2             0.02          sgd  0.504580  0.589925
257          3             0.02          sgd  0.521434  0.543989
108          4             0.36     adaboost  0.524817  0.627112
330          5             0.14          sgd  0.532544  0.548648
233          6             0.18          sgd  0.546124  0.799785
118          7             0.08          sgd  0.548014  0.555926


In [7]:
# with open('ccl_regression_model.pickle', 'rb') as file:
#     automl = pickle.load(file)

In [8]:
predictions = automl.predict(X_test)
mse_value = mean_squared_error(y_test, predictions)
print(mse_value)

76.78500339218657


In [9]:
with open('latex_model.pickle', 'wb') as file:
    pickle.dump(automl, file, protocol=pickle.HIGHEST_PROTOCOL)